In [3]:
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
from urllib.request import urlopen
from bs4 import BeautifulSoup
from haversine import haversine
import pprint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
import pandas as pd
import requests
from sklearn.model_selection import train_test_split
from matplotlib import pyplot

In [4]:
injuries = pd.read_csv("NBA_Injury_Data.csv")
df_cities = pd.read_csv("stadiums.csv") 
df_cities = df_cities[df_cities['League'] == 'NBA']
player = pd.read_csv("joel_embiid_2020_2021.csv")
name = 'Joel Embiid'

In [5]:
daysBetweenRegressor = 0 #calculate game and day played frequency 
minsPlayedRegressor = 0 #take into account mins played yesterday
distTraveledRegressor = 0 #distance traveled to play
injuryRegressor = 0 #effect and severity of injury 
currentInjury = None #name of injury

In [6]:
teams_expanded = {'DAL': 'Dallas Mavericks', 
                'ORL': 'Orlando Magic', 
                'SAS': 'San Antonio Spurs',
                'DEN': 'Denver Nuggets',
                'BKN': 'Brooklyn Nets',
                'BRK': 'Brooklyn Nets',
                'UTA': 'Utah Jazz',
                'WAS': 'Washington Wizards',
                'GSW': 'Golden State Warriors',
                'LAC': 'Los Angeles Clippers',
                'LAL': 'Los Angeles Lakers',
                'MEM': 'Memphis Grizzlies', 
                'MIL': 'Milwaukee Bucks',
                'PHX': 'Phoenix Suns',
                'PHO': 'Phoenix Suns',
                'MIA': 'Miami Heat',
                'IND': 'Indiana Pacers',
                'SAC': 'Sacremento Kings',
                'DET': 'Detroit Pistons',
                'PHI': 'Philadelphia 76ers', 
                'NYK': 'New York Knicks',
                'POR': 'Portland Trail Blazers',
                'OKC': 'Oklahoma City Thunder', 
                'CLE': 'Cleveland Cavaliers',
                'TOR': 'Toronto Raptors',
                'NOP': 'New Orleans Pelicans',
                'CHO': 'Charlotte Hornets',
                'ATL': 'Atlanta Hawks',
                'MIN': 'Minnesota Timberwolves',
                'BOS': 'Boston Celtics',
                'HOU': 'Houston Rockets',
                'CHI': 'Chicago Bulls',
                }

In [7]:
'''
https://towardsdatascience.com/realigning-sports-leagues-with-a-clustering-algorithm-d6e9de9294d0
'''
def dist(lat1, long1, lat2, long2):
    
    lat1, lat2 = np.deg2rad(lat1), np.deg2rad(lat2)
    long1, long2 = np.deg2rad(long1), np.deg2rad(long2)
    EARTH_RADIUS = 3958.8
    return EARTH_RADIUS * np.arccos((np.sin(lat1) * np.sin(lat2)) + 
                          np.cos(lat1) * np.cos(lat2) * np.cos(long2 - long1))

In [8]:
def calcDist(start, end):
    print(start, end)
    start_long = df_cities.loc[df_cities.Team == teams_expanded[start], 'Long'].tolist()[0]
    start_lat = df_cities.loc[df_cities.Team == teams_expanded[start], 'Lat'].tolist()[0]
    end_long = df_cities.loc[df_cities.Team == teams_expanded[end], 'Long'].tolist()[0]
    end_lat = df_cities.loc[df_cities.Team == teams_expanded[end], 'Lat'].tolist()[0]
    print(haversine((start_lat, start_long),(end_lat, end_long), unit='mi'))
    return haversine((start_lat, start_long),(end_lat, end_long), unit='mi')

    

In [9]:
def regressionCalc(injury):
    print("injury detected")
    print(injury)
    currInj = 'standard'
    injRegress = -0.10
    if '(out for season)' in injury:
        currInj = 'EOS'
        injRegress = -1
    elif 'sore' in injury:
        currInj = 'soreness' 
        injRegress = -0.10
    elif 'surgery' in injury:
        currInj = 'surgery recovery' 
        injRegress = -0.10
    elif 'injury' in injury:
        currInj = 'standard injury'
        injRegress = -0.10

    return injRegress

    


In [10]:
def getDayDiff(day1, day2): 
    x = datetime.strptime(day1, "%Y-%m-%d")
    y = datetime.strptime(day2, "%Y-%m-%d")
    return (x-y).days

In [11]:
prevLoc = 'NOP'
prevMins = 0
prevDate = player['Date'].iloc[0]
print(prevDate)
if player['Date'].iloc[0] == None:
    print(player['Home'].iloc[0])
else: 
    print(player['Opp'].iloc[0])



2020-12-23
WAS


In [12]:
cnt = 0
values = []
values.append(1)
for row in player.itertuples():
    print(row.MP[0:2])
    if row.MP[0:2] != 'In' and row.MP[0:2] != 'Di':
        if values[-1] == 0.6: 
            values[-1] = 0.8
        if row.Home == '@':
            dist = calcDist(prevLoc, row.Opp)
            prevLoc = row.Opp
        else:
            dist = calcDist(prevLoc, row.Tm)
            prevLoc = row.Tm


        days = getDayDiff(row.Date, prevDate)
        prevDate = row.Date
        minutes = row.MP[0:2]

        result = injuries[(injuries['Relinquished'] == name) & (injuries['Date'] == row.Date)]
        if result.empty == False:
            injury = regressionCalc(result.Notes)
        else: 
            injury = 0
        print("Distance Traveled: " , dist)
        print("Injury Value: " , injury)
        print("Days Rest: " , days)

        if dist == 0:
            distRegress = 0.10
        elif dist < 1000:
            distRegress = -0.10
        elif dist < 2000:
            distRegress = -0.15
        else: 
            distRegress = -0.20
        if days == 1:
            daysBonus = 0
        elif days == 2:
            daysBonus = 0.05
        elif days == 3: 
            daysBonus = 0.10
        else:
            daysBonus = 0.15
        if re.search('[a-zA-Z]', minutes) == False:
            if int(minutes) < 20:
                minsRegressor = 0
            elif int(minutes) < 30:
                minsRegressor = -0.10
            elif int(minutes) < 40:
                minsRegressor = -0.15
            else: 
                minsRegressor = -0.20
        else:
            minsRegressor = 0
        
        healthValue = values[-1] + (0.33 *  distRegress) + (0.33 *  minsRegressor) + (0.33 * daysBonus) + injury
        if healthValue >= float(1):
            values.append(1.0)
        else:
            values.append(healthValue)
        print(values[-1])
        cnt += 1
    else:
        values.append(0.6)
        print(0.6)

34


AttributeError: 'Pandas' object has no attribute 'Home'

In [ ]:
print(values)
values = values[1:]
print(values)
player['Injury and Fatigue Score'] = values

[1, 1.0, 0.967, 0.9339999999999999, 0.9009999999999999, 0.8844999999999998, 0.9339999999999998, 0.8, 0.8825000000000001, 0.8495, 0.833, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.8, 0.8165, 0.866, 0.8494999999999999, 0.8329999999999999, 0.8329999999999999, 0.8824999999999998, 0.8494999999999998, 0.8989999999999998, 0.8, 0.8825000000000001, 0.8495, 0.866, 0.866, 0.866, 0.8494999999999999, 0.8329999999999999, 0.8494999999999998, 0.8329999999999997, 0.8164999999999997, 0.7999999999999997, 0.7669999999999997, 0.8164999999999997, 0.7669999999999997, 0.7504999999999996, 0.7339999999999995, 0.7834999999999995, 0.7504999999999995, 0.7174999999999995, 0.7669999999999995, 0.8164999999999994, 0.8, 0.8825000000000001, 0.8495, 0.833, 0.8164999999999999, 0.8825, 0.9155, 0.9815, 0.965, 0.9814999999999999, 0.9484999999999999]
[1.0, 0.967, 0.9339999999999999, 0.9009999999999999, 0.8844999999999998, 0.9339999999999998, 0.8, 0.8825000000000001, 0.8495, 0.833, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.8, 0

In [ ]:
print(player['Injury and Fatigue Score'])

0     1.0000
1     0.9670
2     0.9340
3     0.9010
4     0.8845
5     0.9340
6     0.8000
7     0.8825
8     0.8495
9     0.8330
10    0.6000
11    0.6000
12    0.6000
13    0.6000
14    0.6000
15    0.6000
16    0.6000
17    0.6000
18    0.8000
19    0.8165
20    0.8660
21    0.8495
22    0.8330
23    0.8330
24    0.8825
25    0.8495
26    0.8990
27    0.8000
28    0.8825
29    0.8495
30    0.8660
31    0.8660
32    0.8660
33    0.8495
34    0.8330
35    0.8495
36    0.8330
37    0.8165
38    0.8000
39    0.7670
40    0.8165
41    0.7670
42    0.7505
43    0.7340
44    0.7835
45    0.7505
46    0.7175
47    0.7670
48    0.8165
49    0.8000
50    0.8825
51    0.8495
52    0.8330
53    0.8165
54    0.8825
55    0.9155
56    0.9815
57    0.9650
58    0.9815
59    0.9485
Name: Injury and Fatigue Score, dtype: float64
